# check jetson nano

In [1]:
import subprocess
import pkg_resources
import platform
import os, sys
sys.path.append('../')
from utils.cooling_fan import CoolingFan
from utils.battery import Battery

In [2]:
# mkaes only sense if a fan with pwm is connected
cooling_fan = CoolingFan()
print(f'cooling fan speed: {cooling_fan.get_speed()} (max is 255)  \ntemperature: {cooling_fan.get_temp()} degree celsius') 

cooling fan speed: 0 (max is 255)  
temperature: 34.0 degree celsius


In [3]:
# print model
with open('/proc/device-tree/model', 'r') as f:
    print(str(f.read()[:-1]))

NVIDIA Jetson Nano Developer Kit


In [4]:
# network connection(s)
for interface in ['eth0', 'wlan0']:
    operstate = subprocess.check_output('cat /sys/class/net/%s/operstate' % interface, shell=True).decode('ascii')[:-1]
    if operstate == 'up' :  
        cmd = "ifconfig %s | grep -Eo 'inet (addr:)?([0-9]*\.){3}[0-9]*' | grep -Eo '([0-9]*\.){3}[0-9]*' | grep -v '127.0.0.1'" % interface
        ip = subprocess.check_output(cmd, shell=True).decode('ascii')[:-1]
    else:
        ip = ''
    print(f'{interface}\t{operstate}\t{ip}')

eth0	up	192.168.178.90
wlan0	up	192.168.178.71


In [5]:
!/usr/local/cuda/bin/nvcc -V

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Wed_Oct_23_21:14:42_PDT_2019
Cuda compilation tools, release 10.2, V10.2.89


In [6]:
!cat /usr/include/cudnn_version.h | grep CUDNN

#ifndef CUDNN_VERSION_H_
#define CUDNN_VERSION_H_
#define CUDNN_MAJOR 8
#define CUDNN_MINOR 0
#define CUDNN_PATCHLEVEL 0
#define CUDNN_VERSION (CUDNN_MAJOR * 1000 + CUDNN_MINOR * 100 + CUDNN_PATCHLEVEL)
#endif /* CUDNN_VERSION_H */


In [8]:
# input devices (e.g. to detect devicename of a gamepad)
!ls /dev/input/event*
print('###')
!cat /proc/bus/input/devices

/dev/input/event0  /dev/input/event1
###
I: Bus=0000 Vendor=0000 Product=0000 Version=0000
N: Name="tegra-hda HDMI/DP,pcm=3"
P: Phys=ALSA
S: Sysfs=/devices/70030000.hda/sound/card0/input0
U: Uniq=
E: Enabled=0
H: Handlers=event0 
B: PROP=0
B: EV=21
B: SW=140

I: Bus=0019 Vendor=0001 Product=0001 Version=0100
N: Name="gpio-keys"
P: Phys=gpio-keys/input0
S: Sysfs=/devices/gpio-keys/input/input1
U: Uniq=
E: Enabled=0
H: Handlers=kbd event1 
B: PROP=0
B: EV=3
B: KEY=10000000000000 0



In [9]:
# cameras
!ls /dev/video*

/dev/video0


#i2c devices connected to jetson nano

In [10]:
# check i2c bus
#  found 0x3c : display Adafruit_SSD1306 
#  found 0x48 : ads1115 -> analog digital converter to get voltages e.g. the 12V on jetbot platine
#  found 0x60 : motor driver : Adafruit_MotorHAT
#  found 0x70 : ?
for i in range(128):
    address = str(hex(i))
    found_address = os.popen("i2cdetect -y -r 1 " + address + " " + address + " | egrep '" + address[2:] + "' | awk '{print $2}'").read()
    if address[2:] in found_address:
        print('found', address)

found 0x3c
found 0x48
found 0x60
found 0x70


In [11]:
# battery of jetbot

battery = Battery()
print(f'{battery.read_voltage() / 1000:0.1f} V')

12.1 V


# tensorflow / pytorch

In [12]:
import torch, tensorflow

In [13]:
print('python:     ', sys.version.replace('\n', ' '))
print('pytorch:    ', torch.__version__)
print('keras :     ', tensorflow.keras.__version__)
print('tensorflow: ', tensorflow.__version__)
print('CUDNN:      ', torch.backends.cudnn.version())
print('device      ', torch.cuda.get_device_name(torch.cuda.current_device()))
nvidia_jetpack = subprocess.check_output(['apt-cache', 'show', 'nvidia-jetpack']).decode('utf8')
start = nvidia_jetpack.find('Version', 0)
end = nvidia_jetpack.find('\n', start)
print('jetpack     ', nvidia_jetpack[start+9:end])
print(subprocess.check_output(['cat', '/usr/local/cuda/version.txt']).decode('utf8'))

python:      3.6.9 (default, Jan 26 2021, 15:33:00)  [GCC 8.4.0]
pytorch:     1.7.0
keras :      2.4.0
tensorflow:  2.4.0
CUDNN:       8000
device       NVIDIA Tegra X1
jetpack      4.5.1-b17
CUDA Version 10.2.89

